<a href="https://colab.research.google.com/github/thongle28625/ttnt/blob/main/Ch%C3%A0o_m%E1%BB%ABng_b%E1%BA%A1n_%C4%91%E1%BA%BFn_v%E1%BB%9Bi_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import copy
import math
import numpy

X = "X"
O = "O"
EMPTY = None
user = None
ai = None

def initial_state():
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

def player(board):
    count = 0
    for i in board:
        for j in i:
            if j:
                count += 1
    if count % 2 != 0:
        return ai
    return user

def actions(board):
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

def result(board, action):
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    result_board[i][j] = curr_player
    return result_board

def get_horizontal_winner(board):
    # check horizontally
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[i][0]
        if winner_val is not EMPTY:
            all_same = True
            for j in range(1, board_len):
                if board[i][j] != winner_val:
                    all_same = False
                    break
            if all_same:
                return winner_val
    return None

def get_vertical_winner(board):
    # check vertically
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[0][i]
        if winner_val is not EMPTY:
            all_same = True
            for j in range(1, board_len):
                if board[j][i] != winner_val:
                    all_same = False
                    break
            if all_same:
                return winner_val
    return None

def get_diagonal_winner(board):
    # check diagonally
    board_len = len(board)

    # Check the first diagonal (top-left to bottom-right)
    winner_val = board[0][0]
    if winner_val is not EMPTY:
        all_same = True
        for i in range(1, board_len):
            if board[i][i] != winner_val:
                all_same = False
                break
        if all_same:
            return winner_val

    # Check the second diagonal (top-right to bottom-left)
    winner_val = board[0][board_len - 1]
    if winner_val is not EMPTY:
        all_same = True
        for i in range(1, board_len):
            j = board_len - 1 - i
            if board[i][j] != winner_val:
                all_same = False
                break
        if all_same:
            return winner_val

    return None

def winner(board):
    # Check for horizontal winner
    h_winner = get_horizontal_winner(board)
    if h_winner:
        return h_winner
    # Check for vertical winner
    v_winner = get_vertical_winner(board)
    if v_winner:
        return v_winner
    # Check for diagonal winner
    d_winner = get_diagonal_winner(board)
    if d_winner:
        return d_winner
    return None

def terminal(board):
    if winner(board) != None:
        return True
    for i in board:
        for j in i:
            if j == EMPTY:
                return False
    return True

def utility(board):
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v

def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

def minimax(board):
    current_player = player(board)
    if current_player == X:
        max_val = -math.inf
        move = None # Initialize move
        for action in actions(board):
            check = minValue(result(board, action))
            if check > max_val:
                max_val = check
                move = action
        return move
    else:
        min_val = math.inf
        move = None # Initialize move
        for action in actions(board):
            check = maxValue(result(board, action))
            if check < min_val:
                min_val = check
                move = action
        return move

if __name__ == "__main__":
    board = initial_state()
    ai_turn = False
    print("Choose a player (X/O): ")
    user = input()
    if user == "X":
        ai = "O"
    else:
        ai = "X"

    while True:
        game_over = terminal(board)
        playr = player(board)
        if game_over:
            final_winner = winner(board) # Renamed to avoid conflict with function name
            if final_winner is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {final_winner} wins.")
            break
        else:
            # Print current board state for better visualization
            print("Current Board:")
            horizontal_line = "----------"
            print(horizontal_line)
            for row in board:
                formatted_cells = []
                for cell in row:
                    if cell == EMPTY:
                        formatted_cells.append('__')
                    else:
                        formatted_cells.append(str(cell) + '_')
                print("|" + "|".join(formatted_cells) + "|")
            print(horizontal_line)
            print(f"It's {playr}'s turn.")

            if user != playr and not game_over:
                if ai_turn:
                    print("AI is thinking...")
                    move = minimax(board)
                    board = result(board, move)
                    ai_turn = False
                    print("AI made a move:")
            elif user == playr and not game_over:
                ai_turn = True
                print("Enter the position to move (row,col) e.g., 1,2:")
                try:
                    user_input = input()
                    i_str, j_str = user_input.split(',')
                    i = int(i_str.strip())
                    j = int(j_str.strip())

                    if not (0 <= i < 3 and 0 <= j < 3):
                        print("Invalid input: Row and column must be between 0 and 2.")
                        ai_turn = False # Let user try again
                        continue
                    if board[i][j] == EMPTY:
                        board = result(board, (i, j))
                    else:
                        print("Invalid move: Cell is not empty.")
                        ai_turn = False # Let user try again
                except ValueError:
                    print("Invalid input: Please enter numbers for row and column in 'row,col' format.")
                    ai_turn = False # Let user try again
                except IndexError:
                    print("Invalid input: Row and column must be within board limits (0-2). Please use 'row,col' format.")
                    ai_turn = False # Let user try again

            # Print board after a move is made
            print("Board after move:")
            horizontal_line = "----------"
            print(horizontal_line)
            for row in board:
                formatted_cells = []
                for cell in row:
                    if cell == EMPTY:
                        formatted_cells.append('__')
                    else:
                        formatted_cells.append(str(cell) + '_')
                print("|" + "|".join(formatted_cells) + "|")
            print(horizontal_line)
            print("\n")

Choose a player (X/O): 
X
Current Board:
----------
|__|__|__|
|__|__|__|
|__|__|__|
----------
It's X's turn.
Enter the position to move (row,col) e.g., 1,2:
1,1
Board after move:
----------
|__|__|__|
|__|X_|__|
|__|__|__|
----------


Current Board:
----------
|__|__|__|
|__|X_|__|
|__|__|__|
----------
It's O's turn.
AI is thinking...
AI made a move:
Board after move:
----------
|O_|__|__|
|__|X_|__|
|__|__|__|
----------


Current Board:
----------
|O_|__|__|
|__|X_|__|
|__|__|__|
----------
It's X's turn.
Enter the position to move (row,col) e.g., 1,2:


KeyboardInterrupt: Interrupted by user

** Thuật toán Minimax **

1. Các biến toàn cục

In [ ]:
import copy
import math
import numpy
X = "X"
O = "O"
EMPTY = None
user = None
ai = None

X và O: Là ký tự đại diện cho hai người chơi trong trò chơi Tic-Tac-Toe. Một người sẽ chơi bằng dấu "X", người còn lại sẽ chơi bằng dấu "O".

EMPTY: Đại diện cho các ô trống trên bảng (chưa có dấu X hay O).

user và ai: Biến dùng để lưu trữ người chơi và AI. Mỗi khi bắt đầu trò chơi, người chơi sẽ chọn "X" hoặc "O", và AI sẽ tự động nhận ký tự còn lại.

2. Hàm initial_state()

In [ ]:
def initial_state():
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

Hàm này trả về trạng thái ban đầu của bảng Tic-Tac-Toe. Mỗi ô trên bảng bắt đầu là EMPTY (None), tạo thành một bảng 3x3 trống.

3. Hàm player(board)

In [ ]:
def player(board):
    count = 0
    for i in board:
        for j in i:
            if j:
                count += 1
    if count % 2 != 0:
        return ai
    return user

Hàm này xác định người chơi tiếp theo dựa trên số ô đã có dấu (X hoặc O).

Nếu số ô đã được đánh dấu là số lẻ, thì AI sẽ đi tiếp, ngược lại, người chơi sẽ đi tiếp.

4. Hàm actions(board)

In [ ]:
def actions(board):
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res


Hàm này trả về tất cả các hành động hợp lệ (tức là các ô trống) mà người chơi hoặc AI có thể chọn.

res là một tập hợp các chỉ số (i, j) đại diện cho các ô trống trên bảng.

5. Hàm result(board, action)

In [ ]:

def result(board, action):
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    result_board[i][j] = curr_player
    return result_board

Hàm này trả về trạng thái bảng mới sau khi một người chơi hoặc AI thực hiện một nước đi.

action là một tuple (i, j) xác định vị trí mà người chơi chọn.

curr_player là người chơi hiện tại (người hoặc AI), và hàm sẽ đặt dấu của người chơi vào ô (i, j) trên bản sao của bảng (để không thay đổi bảng gốc).